In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_groq import ChatGroq
api_key = os.getenv("GROQ_API_KEY")
llm_model = ChatGroq(groq_api_key=api_key, model="Gemma-7b-It")
llm_model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D944DC8AF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D944DC97B0>, model_name='Gemma-7b-It', groq_api_key=SecretStr('**********'))

In [4]:
from langchain.schema import (AIMessage, HumanMessage, SystemMessage)

In [12]:
speech = """
Ladies and gentlemen, esteemed guests, and fellow enthusiasts of technology,

Today, I have the pleasure of discussing a transformative force that is revolutionizing the way we create, innovate, and interact with technology: Generative Artificial Intelligence, or Generative AI. This fascinating branch of AI is not just reshaping industries but also redefining the boundaries of human creativity and problem-solving.

Generative AI refers to a subset of artificial intelligence that focuses on creating models capable of generating new, previously unseen data that resembles the data it was trained on. These models can produce text, images, music, and even complex designs, often indistinguishable from those created by humans. The underlying technology behind Generative AI includes neural networks, particularly Generative Adversarial Networks (GANs) and transformer models like GPT-4, which power the creation of human-like text and more.

One of the most exciting aspects of Generative AI is its potential to democratize creativity. Historically, creating art, writing literature, composing music, or designing products required years of training and practice. With Generative AI, we can all become creators. For instance, artists can use AI to generate new ideas, musicians can compose unique melodies, and writers can develop engaging narratives, all with the assistance of AI models. This collaboration between human creativity and machine learning is fostering a new era of innovation where the only limit is our imagination.

In the field of healthcare, Generative AI is poised to bring about groundbreaking advancements. AI models can generate realistic synthetic data, which can be used to train other AI systems without compromising patient privacy. This synthetic data can simulate rare diseases, enabling researchers to develop and test new treatments more effectively. Furthermore, Generative AI can aid in drug discovery by predicting molecular structures and potential compounds, significantly speeding up the research and development process.

The impact of Generative AI extends to other industries as well. In fashion, designers are leveraging AI to create unique patterns and styles. In architecture, AI is helping to design innovative structures that are both aesthetically pleasing and structurally sound. In entertainment, AI-generated content is opening new realms of storytelling and interactive experiences.

However, with great power comes great responsibility. The capabilities of Generative AI also raise ethical and societal concerns. The potential for generating deepfakes, misleading information, and other malicious content poses significant challenges. It is imperative that we develop robust frameworks for ethical AI use, ensuring transparency, accountability, and fairness. Collaboration between policymakers, technologists, and ethicists is crucial to navigate these challenges and harness the benefits of Generative AI while mitigating its risks.

Moreover, the rise of Generative AI brings to light the importance of education and upskilling. As AI continues to evolve, so must our workforce. It is essential to equip individuals with the knowledge and skills to work alongside AI, fostering an environment of continuous learning and adaptation. By doing so, we can ensure that the benefits of Generative AI are accessible to all, creating a more inclusive and equitable future.

In conclusion, Generative AI is not just a technological advancement; it is a catalyst for a new wave of human ingenuity. It empowers us to create, innovate, and solve problems in ways we never thought possible. As we embrace this transformative technology, let us do so with a commitment to ethical principles, continuous learning, and a vision for a better, more innovative world. Together, we can harness the power of Generative AI to unlock endless possibilities and shape a future that is both technologically advanced and profoundly human.
"""

In [13]:
chat_message = [
    SystemMessage(content="You are an expert with expertise in summarizing speeches."),
    HumanMessage(content=f"Please provide a short and concise summary of the following speech: \n Text: {speech}")
]


In [14]:
# one way of getting the summary
llm_model(chat_message)

AIMessage(content='**Summary:**\n\nGenerative AI is an innovative branch of AI that creates new, unseen data resembling the training data. It democratizes creativity, enabling individuals to become creators in various fields. In healthcare, it generates realistic synthetic data for research and drug discovery. Across industries, it drives innovation in fashion, architecture, and entertainment. However, ethical considerations and the need for education are paramount to ensure responsible AI use.', response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 754, 'total_tokens': 838, 'completion_time': 0.097852473, 'prompt_time': 0.44162935, 'queue_time': None, 'total_time': 0.539481823}, 'model_name': 'Gemma-7b-It', 'system_fingerprint': 'fp_e3b292b95f', 'finish_reason': 'stop', 'logprobs': None}, id='run-39f4c76f-b1f4-4402-ac88-a25ce6ef9e31-0', usage_metadata={'input_tokens': 754, 'output_tokens': 84, 'total_tokens': 838})

In [15]:
# Create Prompt template for text summary
from langchain.chains import LLMChain
from langchain import PromptTemplate

generic_prompt_template = """
Write a summary of the following speech:
Speech: {speech}
Translate the precise summary to {language}
"""

prompt = PromptTemplate(
    input_variables=['speech', 'language'],
    template=generic_prompt_template
)

prompt

PromptTemplate(input_variables=['language', 'speech'], template='\nWrite a summary of the following speech:\nSpeech: {speech}\nTranslate the precise summary to {language}\n')

In [17]:
complete_prompt = prompt.format(speech=speech, language="Spanish")

In [20]:
llm_chain = LLMChain(llm=llm_model, prompt=prompt)
summary = llm_chain.run({"speech": speech, "language": "Spanish"})
summary

'**Resumen:**\n\nLas mujeres y los hombres distinguidos, invitados y apasionados por la tecnología,\n\nMe siento feliz de hablar sobre una fuerza transformadora que está revolucionando la creación, la innovación y la interacción con la tecnología: el Generativo de Inteligencia Artificial (IA). Este fascinante campo de la IA se basa en el desarrollo de modelos capaces de generar nuevos datos no vistos previamente que se parecen a los datos en los que se han entrenado. Estos modelos pueden producir textos, imágenes, música y diseños complejos, a menudo indistinguibles de los creados por humanos.\n\nEl aspecto más emocionante del Generativo de IA es su potencial para democratizar la creatividad. Históricamente, crear arte, literatura, música o productos requería años de entrenamiento y práctica. Con el Generativo de IA, todos podemos convertirse en creadores. Por ejemplo, los artistas pueden usar IA para generar nuevas ideas, los músicos pueden componer melodías únicas y los escritores pu

### Stuff Document Chain Text Summarization

This is used when the documents are very large and contain many words, i.e the context window size of a model is not able to process large documents together, so we use Stuff Document Chain Text Summarization

In [23]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("CNS_Unit_01.pdf")
docs = loader.load_and_split()
len(docs)

92

In [24]:
prompt_template = """
Write a concise and short summary of the following chapter,
Chapter: {text}
"""

prompt = PromptTemplate(input_variables=['text'], template=prompt_template)

In [26]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm=llm_model, chain_type='stuff', prompt=prompt, verbose=True)
summary_stuff = chain.run(docs[:10])
summary_stuff



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a concise and short summary of the following chapter,
Chapter: Prof. Mandar Diwakar
Asst. Professor
Dept.  of Computer Engineering
Smt. Kashibai Navale College of Engineering, Pune
 1

Computer Network and Security
T.E. Computer Engineering Semester I
Unit I
Introduction To Computer Networks
2 Prof. Mandar P Diwakar, SKNCOE, Pune

What is a Computer Network?
➢Computer Network is a group of 
computers connected with each 
other through wires, optical fibers 
or optical links so that various 
devices can interact with each 
other through a network.
➢The aim of the computer network 
is the sharing of resources 
among various devices.
➢In the case of computer network 
technology, there are several types 
of networks that vary from simple 
to complex level.
3 Prof. Mandar P Diwakar, SKNCOE, Pune

What is need of Computer Network?
1.Information exchange : -To exchange data and inf

"## Summary:\n\nThe chapter discusses the concept of computer networks and their importance in today's digital age. It explores various types of computer networks, including Personal Area Network (PAN), Local Area Network (LAN), Metropolitan Area Network (MAN), Wide Area Network (WAN), and Wireless networks.\n\n**Key points:**\n\n* Computer networks facilitate resource sharing, information exchange, and collaboration among multiple devices.\n* Different types of networks cater to specific communication needs, ranging from personal device connections to wide-area connectivity.\n* Wireless networks eliminate the need for physical cables, offering greater flexibility and convenience.\n\n**The chapter also highlights the benefits of computer networks:**\n\n* Resource sharing and cost reduction\n* Centralized control and security\n* Improved efficiency and productivity\n* Enhanced collaboration and communication\n* Global connectivity and accessibility"

### Map Reduce text summarization

In [30]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
split_docs = text_splitter.split_documents(docs[2:])
split_docs

[Document(metadata={'source': 'CNS_Unit_01.pdf', 'page': 2}, page_content='What is a Computer Network?\n➢Computer Network is a group of \ncomputers connected with each \nother through wires, optical fibers \nor optical links so that various \ndevices can interact with each \nother through a network.\n➢The aim of the computer network \nis the sharing of resources \namong various devices.\n➢In the case of computer network \ntechnology, there are several types \nof networks that vary from simple \nto complex level.\n3 Prof. Mandar P Diwakar, SKNCOE, Pune'),
 Document(metadata={'source': 'CNS_Unit_01.pdf', 'page': 3}, page_content='What is need of Computer Network?\n1.Information exchange : -To exchange data and information between different individual users, it is \nnecessary to interconnect the individual users’ computers .\n2.Resource sharing : -The cost of computer has come down. However, the cost of a laser printer, bulk \nstorage, and large enterprise software remains high. When comp

In [31]:
map_reduce_prompt_template = """
Please summarize the following text,
Chapter: {text}
Summary:
"""

map_reduce_prompt = PromptTemplate(input_variables=['text'], template=map_reduce_prompt_template)

In [33]:
final_prompt = """
Provide the final summary of the the entire text with these important points.
Add a title, start the precise summary with and introduction and provide the summary.
Text: {text}
"""

final_prompt_template = PromptTemplate(input_variables=['text'], template=final_prompt)
final_prompt_template

PromptTemplate(input_variables=['text'], template='\nProvide the final summary of the the entire text with these important points.\nAdd a title, start the precise summary with and introduction and provide the summary.\nText: {text}\n')

In [35]:
summary_chain = load_summarize_chain(
    llm=llm_model, 
    chain_type="map_reduce",
    map_prompt = map_reduce_prompt,
    combine_prompt = final_prompt_template,
    verbose=True
)

In [37]:
map_reduce_summary = summary_chain.run(split_docs[:20])
map_reduce_summary



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the following text,
Chapter: What is a Computer Network?
➢Computer Network is a group of 
computers connected with each 
other through wires, optical fibers 
or optical links so that various 
devices can interact with each 
other through a network.
➢The aim of the computer network 
is the sharing of resources 
among various devices.
➢In the case of computer network 
technology, there are several types 
of networks that vary from simple 
to complex level.
3 Prof. Mandar P Diwakar, SKNCOE, Pune
Summary:

Prompt after formatting:

Please summarize the following text,
Chapter: What is need of Computer Network?
1.Information exchange : -To exchange data and information between different individual users, it is 
necessary to interconnect the individual users’ computers .
2.Resource sharing : -The cost of computer has come down. However, the cost of a laser printer, 

OSError: [WinError 126] The specified module could not be found. Error loading "c:\Yugal Work\Learning\GenAI\Text Summarization\ts-env\lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.